In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np

In [2]:
cv2.__version__

'4.5.2'

In [3]:
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,640)
instr = cv2.imread('instr.png')
instr = cv2.resize(instr, (300, 300))

mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands = 1)
mpDraw = mp.solutions.drawing_utils
tracked_moves = []

start_time = 0
end_time = 0
# modes : draw, erase, hover
mode = "hover"

while True:
    success, img = cap.read()
    img = cv2.flip(img,1)
    
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    # fingers raised
    index = False
    middle = False
    ring = False
    little = False
    
    # added the instruction image
    alpha = 0.5
    img[0:300,0:300,:] = np.uint8(img[0:300,0:300,:]*alpha + instr*(1-alpha))
    
    # get the hand coordinates displayed
    if results.multi_hand_landmarks:
        for handLMs in results.multi_hand_landmarks:
            # get image shape 
            height, width, channel = img.shape
            
            # get the position of the fingertips and bases
            index_tip_y = int(handLMs.landmark[8].y*height)
            index_base_y = int(handLMs.landmark[5].y*height)
            middle_tip_y = int(handLMs.landmark[12].y*height)
            middle_base_y = int(handLMs.landmark[9].y*height)
            ring_tip_y = int(handLMs.landmark[16].y*height)
            ring_base_y = int(handLMs.landmark[13].y*height)
            little_tip_y = int(handLMs.landmark[20].y*height)
            little_base_y = int(handLMs.landmark[17].y*height)
            palm_base_y = int(handLMs.landmark[0].y*height)
            
            
            # get the current hand gesture
            if abs(index_tip_y - palm_base_y) > abs(index_base_y -palm_base_y):
                index = True
            if abs(middle_tip_y - palm_base_y) > abs(middle_base_y -palm_base_y):
                middle = True
            if abs(ring_tip_y - palm_base_y) > abs(ring_base_y -palm_base_y):
                ring = True
            if abs(little_tip_y - palm_base_y) > abs(little_base_y -palm_base_y):
                little = True
            
            # switch to draw
            if index and not middle and not ring and not little:
                cy, cx = index_tip_y, int(handLMs.landmark[8].x*width)
                # if hover mode was on previously
                if mode == "hover":
                    tracked_moves.append([cx, cy, 1])
                    mode = "draw"
                # if erase was the previous state
                elif mode == "erase":
                    tracked_moves.append([cx, cy, 1])
                    mode = "draw"
                # if draw was the previous state
                elif mode == "draw":
                    tracked_moves.append([cx, cy, 0]) 
            
            # switch to erase
            elif index and middle and not ring and not little:
                cy, cx = index_tip_y, int(handLMs.landmark[8].x*width)
                # looped through the tracked moves to get closest points to the currentx, y 
                for no in range(0, len(tracked_moves)):
                    try:
                        # previously tracked moves
                        gx, gy, _ = tracked_moves[no]
                        # if the point is in range of 50 pixels.
                        if np.power(np.power(cx-gx, 2) + np.power(cy-gy, 2), 0.5) < 50:
                            try:
                                tracked_moves[no+1][2] = 1
                                tracked_moves.pop(no)
                            except IndexError:
                                tracked_moves.pop(no)
                                continue
                    except IndexError:
                        continue
                mode = "erase"
                        
            # switch to hover 
            else:
                if mode == "draw":
                    mode = "hover"
                elif mode == "erase":
                    mode = "erase"
            
            mpDraw.draw_landmarks(img, handLMs, mpHands.HAND_CONNECTIONS)
    
    # get list of previous tracked movements
    for move in range(0, len(tracked_moves)):
        try:
            # print the travel between 2 points
            if tracked_moves[move+1][2] == 1:
                continue
            cv2.line(img, (tracked_moves[move][0], tracked_moves[move][1]),
                     (tracked_moves[move+1][0], tracked_moves[move+1][1]), 
                     (255, 0, 0), 15, cv2.FILLED)
        except IndexError:
            cv2.circle(img, (tracked_moves[move][0], tracked_moves[move][1]), 
                     5, (255, 0, 0), cv2.FILLED)
            continue
    # display fps
    end_time = time.time()
    
    fps = 1/(end_time-start_time)
    start_time = end_time
    cv2.putText(img, str(int(fps)), (0, 330), cv2.FONT_HERSHEY_PLAIN, 2, 
                (255,255,0), 3)
    
    # show image
    print(mode, " : ", len(tracked_moves))
    #print(tracked_moves)
    cv2.imshow("ESC to Exit", img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
hover  :  0
draw  :  1
draw  :  2
draw  :  3
draw  :  4
draw  : 